In [1]:
import numpy as np

def heston_mc_price(S0, K, r, T, kappa, theta, sigma, rho, v0,
                    Npaths=20000, Nt=252, option_type='call', seed=42):
    np.random.seed(seed)
    dt = T / Nt
    S = np.full(Npaths, S0, dtype=float)
    v = np.full(Npaths, v0, dtype=float)
    sqrt_dt = np.sqrt(dt)
    for _ in range(Nt):
        Z2 = np.random.normal(size=Npaths)
        Z1_indep = np.random.normal(size=Npaths)
        Z1 = rho * Z2 + np.sqrt(1 - rho**2) * Z1_indep
        S = S * np.exp((r - 0.5 * v) * dt + np.sqrt(np.maximum(v,0.0)) * sqrt_dt * Z1)
        v = v + kappa * (theta - np.maximum(v,0.0)) * dt + sigma * np.sqrt(np.maximum(v,0.0)) * sqrt_dt * Z2
        v = np.maximum(v, 0.0)
    if option_type.lower() == 'call':
        pay = np.maximum(S - K, 0.0)
    else:
        pay = np.maximum(K - S, 0.0)
    price = np.exp(-r*T) * np.mean(pay)
    stderr = np.exp(-r*T) * np.std(pay) / np.sqrt(Npaths)
    return price, stderr

S0 = 188.85
K = 190.0
r = 0.05
T = 1.0
kappa, theta, sigma, rho, v0 = 2.0, 0.05, 0.3, -0.5, 0.05

price, stderr = heston_mc_price(S0, K, r, T, kappa, theta, sigma, rho, v0,
                                Npaths=20000, Nt=252)
print(price, stderr)


20.695760834763064 0.19461543530009912
